# PhageHostPredict: data processing

In this notebook, all the necessary data processing steps are completed in order to run PhageHostPredict models with RBPs and loci. The notebook transforms collections of phage genomes and bacterial genomes to collections of RBPs and loci.

## Libraries & files

In [1]:
import re
import json
import math
import subprocess
import numpy as np
import pandas as pd
import processing_utils as pu
import matplotlib.pyplot as plt
from Bio import SeqIO
from Bio.Seq import Seq
from tqdm import tqdm
from os import listdir
from xgboost import XGBClassifier

In the block below, set the correct directories and locations. Both the phage genomes and bacterial genomes directory should be folders with all genomes present in them. To run Phanotate, the location of the python file should also be provided.

In [41]:
valencia_dir = '/Users/dimi/GoogleDrive/PhD/4_PHAGEHOST_LEARNING/42_DATA/Valencia_data' # general directory
phage_genomes_dir = valencia_dir+'/phages_genomes'
bacterial_genomes_dir = valencia_dir+'/klebsiella_genomes/fasta_files'
phanotate_loc = '/opt/homebrew/Caskroom/miniforge/base/envs/ML1/bin/phanotate.py'
project_dir = '/Users/dimi/Documents/GitHub_Local/PhageHostLearning'
data_suffix = 'Valencia' # choose a suffix for the created data files

## Phage genome processing

#### Phanotate

In [ ]:
phage_files = listdir(phage_genomes_dir)
phage_files.remove('.DS_Store')
#record = SeqIO.read(phages_dir+'/'+phage_files[0], 'fasta')
bar = tqdm(total=len(phage_files), position=0, leave=True)
name_list = []; gene_list = []

for file in phage_files:
    # access PHANOTATE
    file_dir = phage_genomes_dir+'/'+file
    raw_str = phanotate_loc + ' ' + file_dir
    process = subprocess.Popen(raw_str, shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT)
    stdout, stderr = process.communicate()
    std_splits = stdout.split(sep=b'\n')
    std_splits = std_splits[2:] #std_splits.pop(0)
    
    # Save and reload TSV
    temp_tab = open('/Users/Dimi/Desktop/phage_results.tsv', 'wb')
    for split in std_splits:
        split = split.replace(b',', b'') # replace commas for pandas compatibility
        temp_tab.write(split + b'\n')
    temp_tab.close()
    results_orfs = pd.read_csv('/Users/Dimi/Desktop/phage_results.tsv', sep='\t', lineterminator='\n', index_col=False)
    
    # fill up lists accordingly
    name = file.split('_')[0]
    sequence = str(SeqIO.read(file_dir, 'fasta').seq)
    for j, strand in enumerate(results_orfs['FRAME']):
        start = results_orfs['#START'][j]
        stop = results_orfs['STOP'][j]
        
        if strand == '+':
            gene = sequence[start-1:stop]
        else:
            sequence_part = sequence[stop-1:start]
            gene = str(Seq(sequence_part).reverse_complement())
            
        name_list.append(name)
        gene_list.append(gene)
        
    # update progress
    bar.update(1)
bar.close()

In [ ]:
# Export final genes database
genebase = pd.DataFrame(list(zip(name_list, gene_list)), columns=['phage_ID', 'gene_sequence'])
genebase.to_csv(valencia_dir+'/phage_genes'+data_suffix+'.csv', index=False)

#### RBP detection

In the code block below, provide the correct paths to the Pfam file, the trained XGB model, the saved phage genes, and protein embeddings of them.

In [14]:
pfam_file = '/Users/dimi/Documents/GitHub/PhageHostLearning/processing/RBPdetect_phageRBPs.hmm'
xgb_file = 'RBPdetect_xgb_model.json'
#fasta_file = 'data/examples.fasta'
hmmer_path = '/Users/Dimi/hmmer-3.3.1' # e.g. '/Users/Sally/hmmer-3.3.1'
embeddings_file = ... # e.g. 'data/embeddings.csv'

In [15]:
# optionally press database first if not done already
#output, err = pu.hmmpress_python(hmmer_path, pfam_file)

In [18]:
# domain-based RBP detection
N_blocks = ['Phage_T7_tail', 'Tail_spike_N', 'Prophage_tail', 'BppU_N', 'Mtd_N', 
           'Head_binding', 'DUF3751', 'End_N_terminal', 'phage_tail_N', 'Prophage_tailD1', 
           'DUF2163', 'Phage_fiber_2', 'unknown_N0', 'unknown_N1', 'unknown_N2', 'unknown_N3', 'unknown_N4', 
            'unknown_N6', 'unknown_N10', 'unknown_N11', 'unknown_N12', 'unknown_N13', 'unknown_N17', 'unknown_N19', 
            'unknown_N23', 'unknown_N24', 'unknown_N26','unknown_N29', 'unknown_N36', 'unknown_N45', 'unknown_N48', 
            'unknown_N49', 'unknown_N53', 'unknown_N57', 'unknown_N60', 'unknown_N61', 'unknown_N65', 'unknown_N73', 
            'unknown_N82', 'unknown_N83', 'unknown_N101', 'unknown_N114', 'unknown_N119', 'unknown_N122', 
            'unknown_N163', 'unknown_N174', 'unknown_N192', 'unknown_N200', 'unknown_N206', 'unknown_N208']
C_blocks = ['Lipase_GDSL_2', 'Pectate_lyase_3', 'gp37_C', 'Beta_helix', 'Gp58', 'End_beta_propel', 
            'End_tail_spike', 'End_beta_barrel', 'PhageP22-tail', 'Phage_spike_2', 
            'gp12-short_mid', 'Collar', 
            'unknown_C2', 'unknown_C3', 'unknown_C8', 'unknown_C15', 'unknown_C35', 'unknown_C54', 'unknown_C76', 
            'unknown_C100', 'unknown_C105', 'unknown_C112', 'unknown_C123', 'unknown_C179', 'unknown_C201', 
            'unknown_C203', 'unknown_C228', 'unknown_C234', 'unknown_C242', 'unknown_C258', 'unknown_C262', 
            'unknown_C267', 'unknown_C268', 'unknown_C274', 'unknown_C286', 'unknown_C292', 'unknown_C294', 
            'Peptidase_S74', 'Phage_fiber_C', 'S_tail_recep_bd', 'CBM_4_9', 'DUF1983', 'DUF3672']

phage_genes = pd.read_csv(valencia_dir+'/phage_genes'+data_suffix+'.csv')
phage_genes_list = phage_genes['gene_sequence']
phage_genes_ids = phage_genes['phage_ID']

domain_based_detections = pu.RBPdetect_domains(hmmer_path, pfam_file, phage_genes_list, phage_genes_ids, 
                                               N_blocks=N_blocks, C_blocks=C_blocks, detect_others=False)

domain_preds = []
for pid in phage_genes_ids:
    if pid in list(domain_based_detections['identifier']):
        domain_preds.append(1)
    else:
        domain_preds.append(0)

Scanning the genes: 100%|███████████████████| 5657/5657 [01:35<00:00, 59.33it/s]


In [19]:
domain_based_detections

,identifier,DNASeq,N_blocks,C_blocks,N_ranges,C_ranges
0,K39PH122C2,ATGGCATTCAGCTGGCAAGAGTCGGTAAAGCCTGCAGGTACTCAGG...,"[Phage_T7_tail, unknown_N2]","[unknown_C76, unknown_C234, unknown_C286]","[(12, 140), (3, 105)]","[(140, 791), (142, 263), (26, 37)]"
1,K39PH122C2,ATGGCTTTAGTAAAACTTACACGGGTAGCGGAGTGGCTCGGAACGT...,[],"[Pectate_lyase_3, Peptidase_S74]",[],"[(38, 126), (543, 603)]"
2,K60PH164C1,ATGAATTCTCATAACCCGTTTAACACAGGTGGTAGTGGGTGCACCA...,[unknown_N82],[unknown_C105],"[(124, 237)]","[(235, 331)]"
3,K45PH128C2,ATGGCTGATACAACACAATTTGAACAGGCTGTGGATCAGGTTATCG...,[],[unknown_C123],[],"[(66, 115)]"
4,K45PH128C2,ATGGCAATTCCTACTATTCCTCTTCAGATATGGGCTGAAAGTGACG...,[unknown_N48],[],"[(0, 82)]",[]
...,...,...,...,...,...,...
89,K33PH14C2,ATGACTAATATCAAGGCCCGCAAGGGCGGTTCCAGCCAGCCGCGAA...,"[unknown_N12, unknown_N163, unknown_N206, unkn...",[],"[(0, 199), (3, 186), (2, 198), (3, 194), (9, 1...",[]
90,K33PH14C2,ATGGACGCAAACGGCGTTATCACCGGGACTGGTACGAAGTGGCAAT...,[unknown_N11],[],"[(0, 163)]",[]
91,K33PH14C2,CTGGATTCGTGGGCCAATGTTGATGGCGTAGGTTCCATGTACACAA...,[],[DUF1983],[],"[(2, 77)]"
92,K35PH164C3,ATGGATAATGAAATACGCACGGTGGTCACGTACAACCTAGATGGGG...,[Phage_T7_tail],[],"[(0, 117)]",[]


In [ ]:
# ML-based RBP detection
embeddings_df = pd.read_csv(embeddings_file)
embeddings = np.asarray(embeddings_df.iloc[:, 2:])

xgb_saved = XGBClassifier()
xgb_saved.load_model(xgb_file) # load trained model

score_xgb = xgb_saved.predict_proba(embeddings)[:,1]
preds_xgb = (score_xgb > 0.5)*1

In [ ]:
# take union or intersection of both detection methods
...

In [29]:
# make a dataframe and unique protein IDs
proteins = [str(Seq(sequence).translate())[:-1] for sequence in domain_based_detections['DNASeq']]
unique_ids = []
for item in domain_based_detections['identifier']:
    check = 0; count = 0
    while check == 0:
        unique = item+'_RBP'+str(count)
        if unique not in unique_ids:
            unique_ids.append(unique)
            check = 1
        else:
            count += 1
            
RBPs = pd.DataFrame({'phage_nr': domain_based_detections['identifier'], 
                      'host': ['klebsiella_pneumoniae']*len(domain_based_detections['identifier']), 
                      'host_accession': ['-']*len(domain_based_detections['identifier']), 
                      'DNASeq': domain_based_detections['DNASeq'], 
                      'ProteinSeq': proteins, 'N_blocks': domain_based_detections['N_blocks'], 
                      'C_blocks': domain_based_detections['C_blocks'], 'N_ranges': domain_based_detections['N_ranges'], 
                      'C_ranges': domain_based_detections['C_ranges'], 'unique_ID': unique_ids})

In [45]:
RBPs.head()

,phage_nr,host,host_accession,DNASeq,ProteinSeq,N_blocks,C_blocks,N_ranges,C_ranges,unique_ID
0,K39PH122C2,klebsiella_pneumoniae,-,ATGGCATTCAGCTGGCAAGAGTCGGTAAAGCCTGCAGGTACTCAGG...,MAFSWQESVKPAGTQDIQCDIEYLDKSYIHVYLDGAETTAFTWTSS...,"[Phage_T7_tail, unknown_N2]","[unknown_C76, unknown_C234, unknown_C286]","[(12, 140), (3, 105)]","[(140, 791), (142, 263), (26, 37)]",K39PH122C2_RBP0
1,K39PH122C2,klebsiella_pneumoniae,-,ATGGCTTTAGTAAAACTTACACGGGTAGCGGAGTGGCTCGGAACGT...,MALVKLTRVAEWLGTYIHSLSSVQRLLGSKLDDTLCVLDFGAVADY...,[],"[Pectate_lyase_3, Peptidase_S74]",[],"[(38, 126), (543, 603)]",K39PH122C2_RBP1
2,K60PH164C1,klebsiella_pneumoniae,-,ATGAATTCTCATAACCCGTTTAACACAGGTGGTAGTGGGTGCACCA...,MNSHNPFNTGGSGCTNDFRSADPLVDRIIGDAYHVVKEVYLALGNL...,[unknown_N82],[unknown_C105],"[(124, 237)]","[(235, 331)]",K60PH164C1_RBP0
3,K45PH128C2,klebsiella_pneumoniae,-,ATGGCTGATACAACACAATTTGAACAGGCTGTGGATCAGGTTATCG...,MADTTQFEQAVDQVIEDSERLHKVVNGSAIDTVIVEDGSTIPTLRK...,[],[unknown_C123],[],"[(66, 115)]",K45PH128C2_RBP0
4,K45PH128C2,klebsiella_pneumoniae,-,ATGGCAATTCCTACTATTCCTCTTCAGATATGGGCTGAAAGTGACG...,MAIPTIPLQIWAESDVVLPNAHTANKISPIADLWDKGWDLGEKPAC...,[unknown_N48],[],"[(0, 82)]",[],K45PH128C2_RBP1


#### Filter for length & save dataframe

In [37]:
to_delete = [i for i, protein_seq in enumerate(RBPs['ProteinSeq']) if (len(protein_seq)<200 or len(protein_seq)>1500)]
RBPs = RBPs.drop(to_delete)
RBPs = RBPs.reset_index(drop=True)

In [42]:
RBPs.to_csv(valencia_dir+'/RBPbase'+data_suffix+'.csv', index=False)

#### CD-HIT for 3D structure predictions

Here, we cluster the identified RBPs to see how they cluster together. Subsequently, the representative of each of the clusters is subjected to AlphaFold's 3D structure prediction (on high-performance computing infrastructure).

Let's keep it simple: all the already known RBPs (with B-helix) that are representatives are white listed by default (and subsequently all the sequences in those clusters), and all other representatives are subjected to a 3D structure prediction for manual check of B-helix presence. If the B-helix is present, we keep all the sequences in the corresponding clusters.

In [47]:
cdpath = '/Users/dimi/cd-hit-v4.8.1-2019-0228'
known_klebsiella_rbps = '/Users/dimi/GoogleDrive/PhD/4_PHAGEHOST_LEARNING/42_DATA/klebsiella_RBP_data/sequences/klebsiella_RBPs.fasta'

In [48]:
# adding known Klebsiella RBPs
RBPs = pd.read_csv(valencia_dir+'/RBPbase'+data_suffix+'.csv')
combined_fasta = open(valencia_dir+'/RBPs_for_clustering'+data_suffix+'.fasta', 'w')
white_list = open(valencia_dir+'/RBPs_white_list'+data_suffix+'.txt', 'w')

for record in SeqIO.parse(known_klebsiella_rbps, 'fasta'):
    sequence = str(record.seq)
    rec_id = record.id
    combined_fasta.write('>identified_'+rec_id+'\n'+sequence+'\n')
    white_list.write('identified_'+rec_id+'\n')
for i, protein in enumerate(RBPs['ProteinSeq']):
    unique_id = RBPs['unique_ID'][i]
    combined_fasta.write('>'+unique_id+'\n'+protein+'\n')
combined_fasta.close()
white_list.close()

In [54]:
# cluster with CD-HIT
input_file = valencia_dir+'/RBPs_for_clustering'+data_suffix+'.fasta'
output_file = valencia_dir+'/RBPs_clusters'+data_suffix
cdout, cderr = pu.cdhit_python(cdpath, input_file, output_file)

In [59]:
# loop over clusters: if one in RBPs_with_Bhelix -> white list entire cluster
white_list = open(valencia_dir+'/RBPs_white_list'+data_suffix+'.txt')
white_list = white_list.read().split('\n')
output_file = valencia_dir+'/RBPs_clusters'+data_suffix
clusters = open(output_file+'.clstr')

cluster_iter = 0
white_check = 0
cluster_accessions = []
sequences_to_keep = []
for line in clusters.readlines():
    # new cluster
    if line[0] == '>':
        # finish old cluster if not first one
        if (cluster_iter > 0) and (len(cluster_accessions) >= 1):
            if white_check == 1: # an identified RBP in cluster
                sequences_to_keep = sequences_to_keep+cluster_accessions
        # initiate new cluster
        cluster_accessions = []
        cluster_iter += 1
        white_check = 0
    # in a cluster
    else:
        acc = line.split('>')[1].split('...')[0]
        cluster_accessions.append(acc)
        if acc in white_list:
            white_check = 1
        
# finish last cluster
if len(cluster_accessions) >= 1:
    if white_check == 1: # an identified RBP in cluster
        sequences_to_keep = sequences_to_keep+cluster_accessions

In [61]:
len(sequences_to_keep)

149

## Bacterial genome processing

#### Construct dataframe

In [ ]:
klebs_files = listdir(bacterial_genomes_dir)
klebs_files.remove('.DS_Store')
ERS_list = []; desc_list = []; genome_list = []; strain_list = []

for file in klebs_files:
    file_dir = bacterial_genomes_dir+'/'+file
    genome = ''
    descr = ''
    for record in SeqIO.parse(file_dir, 'fasta'):
        descr = descr + '+' + record.description
        genome = genome + str(record.seq)

    ERS_list.append(file.split('_')[0])
    desc_list.append(descr[1:])
    genome_list.append(genome)
    strain_list.append(file.split('_')[1].split('.')[0])

bacterial_genomes = pd.DataFrame({'GI': ['-']*len(klebs_files),'accession': ERS_list,'description':desc_list,
                        'organism': ['Klebsiella pneumoniae']*len(klebs_files),'sequence': genome_list,
                        'sequencing_info': ['-']*len(klebs_files),'strain': strain_list,
                        'number_of_prophages': ['-']*len(klebs_files)})
bacterial_genomes.to_csv(valencia_dir+'/klebsiellaGenomes'+data_suffix+'.csv', index=False)

#### Run Kaptive

In [ ]:
def compute_loci(bacterial_genomes, data_dir, project_dir, database_dir, database_name):
    """
    This function uses kaptive_python to loop over all klebsiella_genomes, construct FASTA files for
    each of them and identify their loci. Importantly, the file_names are later used to construct embeddings,
    so need to be identifiable (accession numbers).
        
    Input:
    - bacterial_genomes: Pandas DataFrame of genomes w/ following column names: 'accession', 'sequence'
    - project_directory: the location of kaptive.py (preferrably in the same project folder)
    - data_dir: location of the database and sequence file(s) to loop over
    - database_name: string of the name of the database (.gbk file)
    """
    kaptive_file_names = []
    serotypes = []
    pbar = tqdm(total=klebsiella_genomes.shape[0])
    
    # loop over klebsiella_genomes
    for i, genome in enumerate(klebsiella_genomes['sequence']):
        #if klebsiella_genomes['number_of_prophages'][i] > 0: # no filter: rows/columns consistent down the road
        acc = list(klebsiella_genomes['accession'])[i]

        # make FASTA file
        file_name = acc+'.fasta'
        fasta = open(data_dir+'/'+file_name, 'w')
        fasta.write('>'+acc+'\n'+genome+'\n')
        fasta.close()

        # run Kaptive
        kaptive_file, _, _ = kaptive_python(project_dir, data_dir, database_name, file_name)
        kaptive_file_names.append(kaptive_file)

        # process json -> proteins in fasta
        results = json.load(open(data_dir+'/kaptive_results.json'))
        serotypes.append(results[0]['Best match']['Type'])
        protein_results = open(data_dir+'/kaptive_results_proteins_'+acc+'.fasta', 'w')
        for gene in results[0]['Locus genes']:
            try:
                name = gene['Reference']['Product']
            except KeyError:
                name = 'unknown'
            protein = gene['Reference']['Protein sequence']
            protein_results.write('>'+name+'\n'+protein[:-1]+'\n')
        protein_results.close()

        # delete FASTA & temp KAPTIVE files
        os.remove(data_dir+'/'+file_name)
        os.remove(data_dir+'/'+file_name+'.ndb')
        os.remove(data_dir+'/'+file_name+'.not')
        os.remove(data_dir+'/'+file_name+'.ntf')
        os.remove(data_dir+'/'+file_name+'.nto')
        os.remove(data_dir+'/kaptive_results.json')

        # update progress
        pbar.update(1)
            
    pbar.close()
    return kaptive_file_names, serotypes

In [ ]:
database = 'Klebsiella_k_locus_primary_reference.gbk'
genomes = pd.read_csv(valencia_dir+'/klebsiellaGenomes'+data_suffix+'.csv')
names, serotypes = pu.compute_loci(genomes, project_dir, valencia_dir, database) # results in the same data_dir
pd.DataFrame(serotypes, columns=['sero']).to_csv(klebsiella_dir+'/kaptive_serotypes_klebsiella_genomes_031221.csv', index=False)


## Interaction matrix construction

In [ ]:
IM = pd.read_excel(valencia_dir+'/klebsiella_phage_host_interactions.xlsx', index_col=0, header=0)
IM_klebsiella = list(IM.index)
IM_phages = [phage.replace(' ', '') for phage in IM.columns]
klebsiellaValencia = pd.read_csv(valencia_dir+'/klebsiellaGenomesValencia.csv')
RBPbaseValencia = pd.read_csv(valencia_dir+'/RBPbaseValencia.csv')

row_names = klebsiellaValencia['accession']
column_names = RBPbaseValencia['unique_ID']
interactions = np.zeros((len(row_names), len(column_names)))
for i, row in enumerate(row_names):
    strain = klebsiellaValencia['strain'][i]
    row_index = IM_klebsiella.index(strain) # index in IM
    for j, col in enumerate(column_names):
        phage = RBPbaseValencia['phage_nr'][j]
        col_index = IM_phages.index(phage) # index in IM
        interactions[i,j] = IM.iloc[row_index, col_index]

IMValencia = pd.DataFrame(interactions, index=row_names, columns=column_names)
IMValencia.to_csv(valencia_dir+'/interactionsValencia.csv')